# Memory profiling - p122

In [1]:
import numpy as np
from imp import reload

%load_ext line_profiler

In [2]:
%%writefile _simulation.py

import numpy as np

def step(*shape):
    # Create random vector with +1 or -1
    return (2 * (np.random.random_sample(shape) < 0.5) - 1)

def simulate(iterations, n=10000):
    s = step(iterations, n)
    x = np.cumsum(s, axis=0)
    bins = np.arange(-30, 30, 1)
    y = np.vstack([np.histogram(x[i,:], bins)[0]
                  for i in range(iterations)])
    return y

Overwriting _simulation.py


In [3]:
import _simulation
reload(_simulation)

<module '_simulation' from '/home2/yannick2/github/Spikes/python/_simulation.py'>

In [4]:
%lprun -T lprof0 -f _simulation.simulate _simulation.simulate(500)


*** Profile printout saved to text file 'lprof0'. 


In [5]:
! cat lprof0

Timer unit: 1e-06 s

Total time: 0.281634 s
File: /home2/yannick2/github/Spikes/python/_simulation.py
Function: simulate at line 8

Line #      Hits         Time  Per Hit   % Time  Line Contents
     8                                           def simulate(iterations, n=10000):
     9         1        78337  78337.0     27.8      s = step(iterations, n)
    10         1        23791  23791.0      8.4      x = np.cumsum(s, axis=0)
    11         1           24     24.0      0.0      bins = np.arange(-30, 30, 1)
    12         1            4      4.0      0.0      y = np.vstack([np.histogram(x[i,:], bins)[0]
    13         1       179477 179477.0     63.7                    for i in range(iterations)])
    14         1            1      1.0      0.0      return y

In [6]:
%load_ext memory_profiler

import gc

gc.collect()

%mprun -T mprof0 -f _simulation.simulate _simulation.simulate(50, int(1e5))
! cat mprof0



*** Profile printout saved to text file mprof0. 
Filename: /home2/yannick2/github/Spikes/python/_simulation.py

Line #    Mem usage    Increment   Line Contents
     8     45.3 MiB      0.0 MiB   def simulate(iterations, n=10000):
     9     88.2 MiB     42.9 MiB       s = step(iterations, n)
    10    126.2 MiB     38.0 MiB       x = np.cumsum(s, axis=0)
    11    126.2 MiB      0.0 MiB       bins = np.arange(-30, 30, 1)
    12    126.4 MiB      0.1 MiB       y = np.vstack([np.histogram(x[i,:], bins)[0]
    13    126.4 MiB      0.0 MiB                     for i in range(iterations)])
    14    126.4 MiB      0.0 MiB       return y

In [7]:
%memit y = np.random.randn(1000,10000)

peak memory: 126.39 MiB, increment: 76.31 MiB


In [8]:
x = np.random.randn(10000, 1000)
%memit y = None
%time %memit x = x * 2
%time %memit x *= 2
%memit x[10,:] = x[11,:] * 2
%memit y = x[11,:] * 2
%memit y = y * 3
%memit x[:] = x[:] * 2

peak memory: 126.44 MiB, increment: -76.29 MiB
peak memory: 202.73 MiB, increment: 76.29 MiB
CPU times: user 47.9 ms, sys: 17.5 ms, total: 65.4 ms
Wall time: 173 ms
peak memory: 126.60 MiB, increment: 0.00 MiB
CPU times: user 29.5 ms, sys: 38.2 ms, total: 67.7 ms
Wall time: 175 ms
peak memory: 126.61 MiB, increment: 0.00 MiB
peak memory: 126.61 MiB, increment: 0.00 MiB
peak memory: 126.61 MiB, increment: 0.00 MiB
peak memory: 202.90 MiB, increment: 76.30 MiB


In [10]:
import scipy 
import scipy.sparse

N = 8000
s = 4000

def bigsparse (N, s):
    x = np.random.randint(0, N, s)
    y = np.random.randint(0, N, s)
    return scipy.sparse.csr_matrix((np.ones(s), (x,y)), (N,N))

sparse1 = None
sparse2 = None
%memit sparse1 = bigsparse(N, s)
foo = None
%time %memit foo = sparse1 * sparse1
%time print((foo != 0).sum())

%memit sparse2 = bigsparse(N, s * 100)
foo = None
%time %memit foo = sparse2 * sparse2
%time print((foo != 0).sum())

%memit nonsparse_m = sparse2.todense()
foo = None
%time %memit foo = nonsparse_m * nonsparse_m
%time print((foo != 0).sum())

peak memory: 406.51 MiB, increment: 0.00 MiB
peak memory: 284.43 MiB, increment: 0.00 MiB
CPU times: user 25.8 ms, sys: 15 ms, total: 40.8 ms
Wall time: 162 ms
2010
CPU times: user 491 µs, sys: 68 µs, total: 559 µs
Wall time: 437 µs
peak memory: 289.00 MiB, increment: 4.56 MiB
peak memory: 484.51 MiB, increment: 195.52 MiB
CPU times: user 207 ms, sys: 13.1 ms, total: 220 ms
Wall time: 322 ms
17084178
CPU times: user 844 ms, sys: 6.01 ms, total: 850 ms
Wall time: 850 ms
peak memory: 1171.51 MiB, increment: 687.00 MiB
peak memory: 1168.25 MiB, increment: 513.16 MiB
CPU times: user 1min 10s, sys: 253 ms, total: 1min 10s
Wall time: 19.3 s
17084178
CPU times: user 272 ms, sys: 21.5 ms, total: 294 ms
Wall time: 98.1 ms
